# Datetime Handler


In [4]:
import pandas as pd
import pytz
from datetime import datetime, timedelta
from typing import Union


In [5]:
def extract_components(dataframe: pd.DataFrame, column: Union[str, int]) -> pd.DataFrame:
    '''Extract date components (year, month, day, hour, minute, second) into separate columns.'''
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_datetime64_any_dtype(dataframe[column]), f"Column '{column} is not datetime type'"
    
    df_copy = dataframe.copy()
    df_copy['year'] = dataframe.iloc[:, column].dt.year
    df_copy['month'] = dataframe.iloc[:, column].dt.month
    df_copy['day'] = dataframe.iloc[:, column].dt.day
    df_copy['hour'] = dataframe.iloc[:, column].dt.hour
    df_copy['minute'] = dataframe.iloc[:, column].dt.minute
    df_copy['second'] = dataframe.iloc[:, column].dt.second
    return df_copy


In [6]:
def reformat_date(dataframe: pd.DataFrame, column: Union[str, int]) -> pd.DataFrame:
    '''Reformat the datetime objects in the specified column to a different string format.'''
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_datetime64_any_dtype(dataframe[column]), f"Column '{column} is not datetime type'"
    
    df_copy = dataframe.copy()
    df_copy.iloc[:, column] = df_copy.iloc[:, column].apply(lambda x: x.strftime('%d-%m-%Y %H:%M:%S') if pd.notnull(x) else x)
    return df_copy


In [7]:
def calculate_datetime_differences(dataframe: pd.DataFrame, column: Union[str, int]) -> pd.DataFrame:
    '''Calculate the difference between consecutive datetime entries in the specified column.'''
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_datetime64_any_dtype(dataframe[column]), f"Column '{column} is not datetime type'"
    
    df_copy = dataframe.copy()
    df_copy['time_diff'] = df_copy.iloc[:, column].diff().dt.total_seconds()
    return df_copy


In [8]:
def convert_datetime_to_different_timezones(dataframe: pd.DataFrame, column: Union[str, int], from_tz='UTC', to_tz='America/New_York') -> pd.DataFrame:
    '''Convert datetime objects from one timezone to another.'''
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_datetime64_any_dtype(dataframe[column]), f"Column '{column} is not datetime type'"
    
    df_copy = dataframe.copy()
    from_zone = pytz.timezone(from_tz)
    to_zone = pytz.timezone(to_tz)
    def convert_timezone(dt):
        if pd.notnull(dt):
            dt = from_zone.localize(dt) if dt.tzinfo is None else dt
            return dt.astimezone(to_zone)
        return dt
    df_copy.iloc[:, column] = df_copy.iloc[:, column].apply(convert_timezone)
    return df_copy


In [11]:
def shift_time(dataframe: pd.DataFrame, column: Union[str, int], shift_value=1, unit='days') -> pd.DataFrame:
    '''Shift the datetime values in the specified column by a given amount.'''
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    assert pd.api.types.is_datetime64_any_dtype(dataframe[column]), f"Column '{column} is not datetime type'"
    
    df_copy = dataframe.copy()
    df_copy[column] = df_copy[column] + pd.to_timedelta(shift_value, unit=unit)
    return df_copy


### Test


In [12]:
df = pd.read_csv("dataset/movies.csv")
df['Release Date'] = pd.to_datetime(df['Release Date'], errors='coerce')
shift_time(df, 'Release Date', 1)


,Title,Director,Genre,Release Date,Duration,Rating
0,Key entire popular.,Anthony Becker,Horror,1981-05-13,102,6.8
1,Gun husband reveal.,William Johnson,Documentary,2016-06-14,92,7.6
2,Crime cover.,Amy Le,Drama,1988-03-23,144,5.5
3,Challenge.,Andrea Martinez,Romance,2013-04-02,161,2.0
4,Close study.,Michael Rodgers,Fantasy,2012-10-19,177,3.7
...,...,...,...,...,...,...
29995,Daughter.,Richard Nelson,Romance,2007-03-13,177,8.0
29996,Simply.,Jeffrey Hatfield,Fantasy,2011-08-17,126,5.7
29997,Also authority nor.,Ryan Brown,Action,1998-05-08,73,4.9
29998,Total report upon.,Melissa Stephenson,Comedy,2008-06-07,145,6.9
